In [1]:
import io
import random
import string
import numpy as np
import bs4 as bs

import requests 

##regular expression
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

#nltk.download('punkt') 
#nltk.download('wordnet') 

In [3]:
##returns the object which contains
r=requests.get('https://en.wikipedia.org/wiki/Naruto')

#raw_html contains photos, special characters which is not needed
raw_html=r.text 

In [4]:
##cleaning up
corpus_html=bs.BeautifulSoup(raw_html)

#extracting paragraphs only from html
corpus_paras=corpus_html.find_all('p')
corpus_text='' 

#concatenating all the paras
for para in corpus_paras:
    corpus_text+=para.text

##lowering all the text
corpus_text=corpus_text.lower()


In [5]:
corpus_text

'\nnaruto (japanese: naruto（ナルト）) is a japanese manga series written and illustrated by masashi kishimoto. it tells the story of naruto uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the hokage, the leader of his village. the story is told in two parts – the first set in naruto\'s pre-teen years, and the second in his teens. the series is based on two one-shot manga by kishimoto: karakuri (1995), which earned kishimoto an honorable mention in shueisha\'s monthly hop step award the following year, and naruto (1997).\nnaruto was serialized in shueisha\'s magazine, weekly shōnen jump from 1999 to 2014, and released in tankōbon (book) form in 72 volumes. the manga was adapted into an anime television series produced by pierrot and aniplex, which broadcast 220 episodes in japan from 2002 to 2007; the english adaptation of the series aired on cartoon network and ytv from 2005 to 2009. naruto: shippuden, a sequel to the original series, premiered in japan i

In [6]:
corpus_text = re.sub(r'\[[0-9]*\]', ' ', corpus_text)
corpus_text = re.sub(r'\s+', ' ', corpus_text)

In [7]:
raw=corpus_text
raw = raw.lower()#lowercase

In [8]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [10]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [11]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response



In [12]:
flag=True
print("NarutoBot: My name is Naruto Uzumaki. I will answer your queries about me. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Naruto: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Naruto: "+greeting(user_response))
            else:
                print("Naruto: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Naruto: Bye! take care..")

NarutoBot: My name is Naruto Uzumaki. I will answer your queries about me. If you want to exit, type Bye!
hi
Naruto: hi there
village
Naruto: when kishimoto was creating the primary setting of the naruto manga, he concentrated initially on the designs for the village of konoha.
bye
Naruto: Bye! take care..
